In [1]:
!pip install pycaret
!pip install optuna
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:

In [2]:
pip install --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.1.0 requires scipy~=1.10.1, but you have scipy 1.11.2 which is incompatible.


In [3]:
pip install scipy

In [4]:
pip install yellowbrick --upgrade

In [5]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from pycaret.classification import *

from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import accuracy_score

In [9]:
%cd /content/drive/MyDrive/BigContest^>^/

/content/drive/MyDrive/BigContest^>^


In [40]:
# 데이터 불러오기
data = pd.read_csv("raw_data_final.csv", encoding="cp949")

In [41]:
data['INDUSTRY_CD_label'] = data['INDUSTRY_CD'].str[1:].astype('int')

In [42]:
train = data[(data['train']==1)&(data['DATA_CRTR_YM']<=202210)].reset_index()
valid = data[(data['train']==1)&(data['DATA_CRTR_YM']<=202212)&(data['DATA_CRTR_YM']>=202211)].reset_index()
test = data[data['train']==0].reset_index()

In [43]:
train.drop(columns=['전체점포수', '전체 임대료', '65세이상_남', '65세이상_여','index','train'], axis=1, inplace=True)
valid.drop(columns=['전체점포수', '전체 임대료', '65세이상_남', '65세이상_여','index','train'], axis=1, inplace=True)
test.drop(columns=['전체점포수', '전체 임대료', '65세이상_남', '65세이상_여','index','train'], axis=1, inplace=True)

# **업종별로 나눈 데이터 생성하기**

In [62]:
train8 = train[train['INDUSTRY_CD_label'] == 8]
train9 = train[train['INDUSTRY_CD_label'] == 9]
train10 = train[train['INDUSTRY_CD_label'] == 10]
train11 = train[train['INDUSTRY_CD_label'] == 11]

In [63]:
valid8 = valid[valid['INDUSTRY_CD_label'] == 8]
valid9 = valid[valid['INDUSTRY_CD_label'] == 9]
valid10 = valid[valid['INDUSTRY_CD_label'] == 10]
valid11 = valid[valid['INDUSTRY_CD_label'] == 11]

In [64]:
test8 = test[test['INDUSTRY_CD_label'] == 8]
test9 = test[test['INDUSTRY_CD_label'] == 9]
test10 = test[test['INDUSTRY_CD_label'] == 10]
test11 = test[test['INDUSTRY_CD_label'] == 11]

In [65]:
columns = ['프랜차이즈점포수', '일반점포수', '길단위유동인구', '주거인구', '직장인구', '개업수', '폐업수', '개업률', '폐업률',
           '1층 임대료', '1층 외 임대료', '생활물가지수', '부동산거래대비유동인구', '공실률대비매매가임대료', '젠트리피케이션',
           '지하철개수', '스타벅스개수', '65세이상', '출근시간_승차수', '출근시간_하차수', '주말_하차수']

In [66]:
X_train8 = train8[columns]
X_train9 = train9[columns]
X_train10 = train10[columns]
X_train11 = train11[columns]

In [67]:
y_train8 = train8['SLS_GRD']
y_train9 = train9['SLS_GRD']
y_train10 = train10['SLS_GRD']
y_train11 = train11['SLS_GRD']

In [68]:
X_valid8 = valid8[columns]
X_valid9 = valid9[columns]
X_valid10 = valid10[columns]
X_valid11 = valid11[columns]

In [69]:
y_valid8 = valid8['SLS_GRD']
y_valid9 = valid9['SLS_GRD']
y_valid10 = valid10['SLS_GRD']
y_valid11 = valid11['SLS_GRD']

In [70]:
X_test8 = test8[columns]
X_test9 = test9[columns]
X_test10 = test10[columns]
X_test11 = test11[columns]

In [71]:
features_to_scale = X_train8.columns
scaler8 = StandardScaler()
scaler8.fit(X_train8[features_to_scale])
X_train8[features_to_scale] = scaler8.transform(X_train8[features_to_scale])
X_valid8[features_to_scale] = scaler8.transform(X_valid8[features_to_scale])
X_test8[features_to_scale] = scaler8.transform(X_test8[features_to_scale])

In [72]:
scaler9 = StandardScaler()
scaler9.fit(X_train9[features_to_scale])
X_train9[features_to_scale] = scaler9.transform(X_train9[features_to_scale])
X_valid9[features_to_scale] = scaler9.transform(X_valid9[features_to_scale])
X_test9[features_to_scale] = scaler9.transform(X_test9[features_to_scale])

In [73]:
scaler10 = StandardScaler()
scaler10.fit(X_train10[features_to_scale])
X_train10[features_to_scale] = scaler10.transform(X_train10[features_to_scale])
X_valid10[features_to_scale] = scaler10.transform(X_valid10[features_to_scale])
X_test10[features_to_scale] = scaler10.transform(X_test10[features_to_scale])

In [74]:
scaler11 = StandardScaler()
scaler11.fit(X_train11[features_to_scale])
X_train11[features_to_scale] = scaler11.transform(X_train11[features_to_scale])
X_valid11[features_to_scale] = scaler11.transform(X_valid11[features_to_scale])
X_test11[features_to_scale] = scaler11.transform(X_test11[features_to_scale])

In [76]:
X_train8['대학교여부'] = train8['대학교여부']
X_valid8['대학교여부'] = valid8['대학교여부']
X_test8['대학교여부'] = test8['대학교여부']

X_train9['대학교여부'] = train9['대학교여부']
X_valid9['대학교여부'] = valid9['대학교여부']
X_test9['대학교여부'] = test9['대학교여부']

X_train10['대학교여부'] = train10['대학교여부']
X_valid10['대학교여부'] = valid10['대학교여부']
X_test10['대학교여부'] = test10['대학교여부']

X_train11['대학교여부'] = train11['대학교여부']
X_valid11['대학교여부'] = valid11['대학교여부']
X_test11['대학교여부'] = test11['대학교여부']

In [84]:
train_df8 = pd.concat([X_train8, y_train8], axis = 1)
valid_df8 = pd.concat([X_valid8, y_valid8], axis = 1)

train_df9 = pd.concat([X_train9, y_train9], axis = 1)
valid_df9 = pd.concat([X_valid9, y_valid9], axis = 1)

train_df10 = pd.concat([X_train10, y_train10], axis = 1)
valid_df10 = pd.concat([X_valid10, y_valid10], axis = 1)

train_df11 = pd.concat([X_train11, y_train11], axis = 1)
valid_df11 = pd.concat([X_valid11, y_valid11], axis = 1)

In [82]:
train_df8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46958 entries, 98 to 961080
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   프랜차이즈점포수     46958 non-null  float64
 1   일반점포수        46958 non-null  float64
 2   길단위유동인구      46958 non-null  float64
 3   주거인구         46958 non-null  float64
 4   직장인구         46958 non-null  float64
 5   개업수          46958 non-null  float64
 6   폐업수          46958 non-null  float64
 7   개업률          46958 non-null  float64
 8   폐업률          46958 non-null  float64
 9   1층 임대료       46958 non-null  float64
 10  1층 외 임대료     46958 non-null  float64
 11  생활물가지수       46958 non-null  float64
 12  부동산거래대비유동인구  46958 non-null  float64
 13  공실률대비매매가임대료  46958 non-null  float64
 14  젠트리피케이션      46958 non-null  float64
 15  지하철개수        46958 non-null  float64
 16  스타벅스개수       46958 non-null  float64
 17  65세이상        46958 non-null  float64
 18  출근시간_승차수     46958 non-null  float64
 19  출근

In [85]:
valid_df8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10571 entries, 34 to 194876
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   프랜차이즈점포수     10571 non-null  float64
 1   일반점포수        10571 non-null  float64
 2   길단위유동인구      10571 non-null  float64
 3   주거인구         10571 non-null  float64
 4   직장인구         10571 non-null  float64
 5   개업수          10571 non-null  float64
 6   폐업수          10571 non-null  float64
 7   개업률          10571 non-null  float64
 8   폐업률          10571 non-null  float64
 9   1층 임대료       10571 non-null  float64
 10  1층 외 임대료     10571 non-null  float64
 11  생활물가지수       10571 non-null  float64
 12  부동산거래대비유동인구  10571 non-null  float64
 13  공실률대비매매가임대료  10571 non-null  float64
 14  젠트리피케이션      10571 non-null  float64
 15  지하철개수        10571 non-null  float64
 16  스타벅스개수       10571 non-null  float64
 17  65세이상        10571 non-null  float64
 18  출근시간_승차수     10571 non-null  float64
 19  출근

# **A08 모델링**

In [86]:
set8 = setup(data = train_df8, test_data = valid_df8, target = 'SLS_GRD',
             index = False, categorical_features = ['대학교여부'],
             fold = 5, normalize = False, session_id = 0)

,Description,Value
0,Session id,0
1,Target,SLS_GRD
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(57529, 23)"
5,Transformed data shape,"(57529, 23)"
6,Transformed train set shape,"(46958, 23)"
7,Transformed test set shape,"(10571, 23)"
8,Ordinal features,1
9,Numeric features,21


In [87]:
top10 = compare_models(sort = 'acc', n_select = 10)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7243,0.8758,0.7213,0.7218,0.7206,0.6378,0.6382,8.7440
rf,Random Forest Classifier,0.7177,0.8799,0.7163,0.7136,0.7134,0.6284,0.6289,14.6700
dt,Decision Tree Classifier,0.6346,0.7621,0.6307,0.6344,0.6324,0.5231,0.5232,1.1100
catboost,CatBoost Classifier,0.5443,0.8035,0.5537,0.5447,0.5418,0.3938,0.3958,57.5540
xgboost,Extreme Gradient Boosting,0.5382,0.7971,0.5416,0.5400,0.5339,0.3874,0.3893,30.2580
lightgbm,Light Gradient Boosting Machine,0.5136,0.7800,0.5212,0.5214,0.5097,0.3516,0.3548,12.9200
knn,K Neighbors Classifier,0.4948,0.7637,0.4868,0.5024,0.4838,0.3336,0.3364,4.6320
gbc,Gradient Boosting Classifier,0.3940,0.6804,0.4125,0.4044,0.3843,0.1796,0.1850,56.2460
ada,Ada Boost Classifier,0.3389,0.6222,0.3563,0.3471,0.3265,0.1034,0.1060,4.4360
lda,Linear Discriminant Analysis,0.3359,0.6110,0.3469,0.3071,0.3107,0.0972,0.1019,0.6980


Processing:   0%|          | 0/78 [00:00<?, ?it/s]

In [ ]:
model = ExtraTreesClassifier(n_estimators=100, random_state=2023)
model.fit(X_train8, y_train8)

print("훈련 세트 정확도: {:.3f}".format(model.score(X_train8, y_train8)))
print("테스트 세트 정확도: {:.3f}".format(model.score(X_valid8, y_valid8)))

훈련 세트 정확도: 0.994
테스트 세트 정확도: 0.749


In [101]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(n_estimators=100, random_state=2023)
model.fit(X_train8, y_train8)
y_pred8=model.predict(X_valid8)

from sklearn.metrics import mean_squared_error
mse8=mean_squared_error(y_valid8, y_pred8)

print('A08 MSE : {:.4f}'.format(mse8))

A08 MSE : 0.4170


In [92]:
model8=model.fit(X_train8, y_train8)
print(model8)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=None, oob_score=False,
                     random_state=2023, verbose=0, warm_start=False)


In [90]:
importances = model.feature_importances_

# 각 특성의 중요도를 출력한다
for feature, importance in zip(X_train8.columns, importances):
    print(f"{feature}: {importance}")

프랜차이즈점포수: 0.027512613025748065
일반점포수: 0.024195872506324975
길단위유동인구: 0.025511285928146422
주거인구: 0.024431004922987696
직장인구: 0.02615005018068547
개업수: 0.015300020547397377
폐업수: 0.013994692795373393
개업률: 0.01658690876528225
폐업률: 0.015110211634112934
1층 임대료: 0.02232106874209158
1층 외 임대료: 0.021080106063480493
생활물가지수: 0.043021216464996195
부동산거래대비유동인구: 0.2207948016564378
공실률대비매매가임대료: 0.20828978358714206
젠트리피케이션: 0.16321463558726848
지하철개수: 0.053650087563307365
스타벅스개수: 0.00615700204646958
65세이상: 0.056580953395136274
출근시간_승차수: 0.0005410583266482691
출근시간_하차수: 0.0004973166723830726
주말_하차수: 0.0004445636540308575
대학교여부: 0.014614745934549318


# **A09 업종 모델**

In [96]:
set9 = setup(data = train_df9, test_data = valid_df9, target = 'SLS_GRD',
             index = False, categorical_features = ['대학교여부'],
             fold = 5, normalize = False, session_id = 0)

,Description,Value
0,Session id,0
1,Target,SLS_GRD
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(4438, 23)"
5,Transformed data shape,"(4438, 23)"
6,Transformed train set shape,"(3696, 23)"
7,Transformed test set shape,"(742, 23)"
8,Ordinal features,1
9,Numeric features,21


In [97]:
top10 = compare_models(sort = 'acc', n_select = 10)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.6821,0.8557,0.6770,0.6808,0.6783,0.5840,0.5842,0.7140
rf,Random Forest Classifier,0.6815,0.8574,0.6779,0.6790,0.6779,0.5829,0.5831,1.0780
catboost,CatBoost Classifier,0.6772,0.8676,0.6743,0.6761,0.6738,0.5765,0.5768,17.8180
xgboost,Extreme Gradient Boosting,0.6726,0.8594,0.6697,0.6725,0.6694,0.5704,0.5711,3.5360
lightgbm,Light Gradient Boosting Machine,0.6721,0.8639,0.6702,0.6724,0.6690,0.5692,0.5699,4.0980
dt,Decision Tree Classifier,0.6125,0.7473,0.6091,0.6123,0.6101,0.4947,0.4950,0.1580
gbc,Gradient Boosting Classifier,0.5944,0.8195,0.5943,0.5955,0.5895,0.4637,0.4657,6.9840
knn,K Neighbors Classifier,0.5484,0.7955,0.5394,0.5630,0.5420,0.4072,0.4095,0.6500
ada,Ada Boost Classifier,0.3642,0.6372,0.3656,0.3600,0.3593,0.1625,0.1633,1.0700
lr,Logistic Regression,0.3574,0.6257,0.3635,0.3229,0.3334,0.1323,0.1354,2.0160


Processing:   0%|          | 0/78 [00:00<?, ?it/s]

In [100]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(n_estimators=100, random_state=2023)
model.fit(X_train9, y_train9)
y_pred9=model.predict(X_valid9)

from sklearn.metrics import mean_squared_error
mse9=mean_squared_error(y_valid9, y_pred9)

print('A09 MSE : {:.4f}'.format(mse9))

A09 MSE : 0.5795


# **A10 업종 모델**

In [102]:
set10 = setup(data = train_df10, test_data = valid_df10, target = 'SLS_GRD',
             index = False, categorical_features = ['대학교여부'],
             fold = 5, normalize = False, session_id = 0)

,Description,Value
0,Session id,0
1,Target,SLS_GRD
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(5094, 23)"
5,Transformed data shape,"(5094, 23)"
6,Transformed train set shape,"(4189, 23)"
7,Transformed test set shape,"(905, 23)"
8,Ordinal features,1
9,Numeric features,21


In [103]:
top10 = compare_models(sort = 'acc', n_select = 10)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7219,0.8984,0.7122,0.7141,0.7125,0.6360,0.6363,1.0580
et,Extra Trees Classifier,0.7183,0.8865,0.7088,0.7104,0.7091,0.6314,0.6316,0.5780
lightgbm,Light Gradient Boosting Machine,0.7176,0.8996,0.7114,0.7096,0.7091,0.6295,0.6301,5.8560
catboost,CatBoost Classifier,0.7116,0.9029,0.7052,0.7031,0.7023,0.6214,0.6222,20.2900
xgboost,Extreme Gradient Boosting,0.7083,0.8961,0.7022,0.7008,0.7002,0.6174,0.6180,3.8820
dt,Decision Tree Classifier,0.6732,0.7884,0.6678,0.6681,0.6676,0.5729,0.5731,0.1980
gbc,Gradient Boosting Classifier,0.6307,0.8622,0.6274,0.6280,0.6219,0.5120,0.5137,6.3460
knn,K Neighbors Classifier,0.6044,0.8386,0.5912,0.6020,0.5879,0.4789,0.4809,0.2740
lr,Logistic Regression,0.3960,0.6741,0.4101,0.4086,0.3780,0.1852,0.1897,0.2280
ridge,Ridge Classifier,0.3956,0.0000,0.4117,0.3573,0.3754,0.1823,0.1874,0.1500


Processing:   0%|          | 0/78 [00:00<?, ?it/s]

In [105]:
from sklearn.ensemble import RandomForestClassifier

rf10 = RandomForestClassifier(random_state = 0)
rf10.fit(X_train10, y_train10)
y_pred10 = rf10.predict(X_valid10)

from sklearn.metrics import mean_squared_error
mse10=mean_squared_error(y_valid10, y_pred10)

print('A10 MSE : {:.4f}'.format(mse))

A10 MSE : 0.5657


# **A11 업종 모델**

In [106]:
set11 = setup(data = train_df11, test_data = valid_df11, target = 'SLS_GRD',
             index = False, categorical_features = ['대학교여부'],
             fold = 5, normalize = False, session_id = 0)

,Description,Value
0,Session id,0
1,Target,SLS_GRD
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(49329, 23)"
5,Transformed data shape,"(49329, 23)"
6,Transformed train set shape,"(40960, 23)"
7,Transformed test set shape,"(8369, 23)"
8,Ordinal features,1
9,Numeric features,21


In [107]:
top10 = compare_models(sort = 'acc', n_select = 10)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7329,0.8834,0.7299,0.7337,0.7309,0.6497,0.6500,5.8020
rf,Random Forest Classifier,0.7211,0.8887,0.7186,0.7222,0.7190,0.6335,0.6339,7.7860
dt,Decision Tree Classifier,0.6332,0.7591,0.6319,0.6317,0.6317,0.5204,0.5204,0.5540
catboost,CatBoost Classifier,0.5865,0.8389,0.5870,0.5930,0.5834,0.4510,0.4525,52.0880
xgboost,Extreme Gradient Boosting,0.5770,0.8323,0.5784,0.5845,0.5746,0.4381,0.4396,26.5100
lightgbm,Light Gradient Boosting Machine,0.5545,0.8178,0.5557,0.5647,0.5513,0.4067,0.4086,14.1500
knn,K Neighbors Classifier,0.5392,0.7998,0.5370,0.5432,0.5318,0.3925,0.3946,3.8060
gbc,Gradient Boosting Classifier,0.4391,0.7263,0.4426,0.4611,0.4285,0.2434,0.2469,44.7660
lr,Logistic Regression,0.3719,0.6491,0.3774,0.3924,0.3474,0.1431,0.1468,1.8680
ada,Ada Boost Classifier,0.3692,0.6514,0.3730,0.3766,0.3589,0.1510,0.1528,2.9660


Processing:   0%|          | 0/78 [00:00<?, ?it/s]

In [108]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(n_estimators=100, random_state=2023)
model.fit(X_train11, y_train11)
y_pred11=model.predict(X_valid11)

from sklearn.metrics import mean_squared_error
mse11=mean_squared_error(y_valid11, y_pred11)

print('A11 MSE : {:.4f}'.format(mse11))

A11 MSE : 0.3855


In [109]:
mse8=mean_squared_error(y_valid8, y_pred8)
print('A08 MSE : {:.4f}'.format(mse8))

mse9=mean_squared_error(y_valid9, y_pred9)
print('A09 MSE : {:.4f}'.format(mse9))

mse10=mean_squared_error(y_valid10, y_pred10)
print('A10 MSE : {:.4f}'.format(mse10))

mse11=mean_squared_error(y_valid11, y_pred11)
print('A11 MSE : {:.4f}'.format(mse11))

A08 MSE : 0.4170
A09 MSE : 0.5795
A10 MSE : 0.5657
A11 MSE : 0.3855
